# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests



# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Pedernales,18.0384,-71.7440,28.07,62,6,4.36,DO,1683414197
1,1,Leoben,47.3461,15.1257,10.47,80,100,0.68,AT,1683414198
2,2,Yellowknife,62.4560,-114.3525,19.29,30,20,9.26,CA,1683414198
3,3,Edinburgh of the Seven Seas,-37.0676,-12.3116,15.37,75,100,5.32,SH,1683414198
4,4,Waitangi,-43.9535,-176.5597,16.01,89,100,2.68,NZ,1683414198


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )


# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
city_data_df.describe()

,City_ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,562.000000,562.000000,562.000000,562.000000,562.000000,562.000000,562.000000,5.620000e+02
mean,280.500000,17.791035,18.483280,17.439911,71.256228,58.839858,3.689769,1.683414e+09
std,162.379699,32.645980,90.415707,10.126419,21.604495,38.915915,2.445332,6.785340e+01
min,0.000000,-54.800000,-176.559700,-20.780000,7.000000,0.000000,0.000000,1.683414e+09
25%,140.250000,-9.229375,-61.385350,10.935000,61.000000,20.000000,1.880000,1.683414e+09
50%,280.500000,21.115550,24.861000,18.375000,77.000000,72.500000,3.090000,1.683414e+09
75%,420.750000,46.168500,96.115625,26.360000,88.000000,99.000000,4.950000,1.683414e+09
max,561.000000,78.218600,178.441500,37.490000,100.000000,100.000000,13.490000,1.683414e+09


In [13]:
# Narrow down cities that fit criteria and drop any results with null values

sorted_df=city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Cloudiness"] < 2)
                                       & (city_data_df["Wind Speed"] < 3.5)
                                      ]

# Drop any rows with null values
sorted_df.dropna()

# Display sample data
sorted_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
74,74,Inhambane,-23.8650,35.3833,21.69,81,0,2.69,MZ,1683414207
341,341,Dibba Al-Hisn,25.6196,56.2729,24.10,43,0,0.51,AE,1683414232
407,407,Mymensingh,24.7500,90.4000,25.66,72,0,2.36,BD,1683414238
468,468,Manicoré,-5.8092,-61.3003,26.97,89,0,1.03,BR,1683414244
478,478,Parauapebas,-6.0675,-49.9022,25.99,88,0,2.06,BR,1683414239
499,499,Unaí,-16.3575,-46.9061,22.60,53,1,1.25,BR,1683414237
527,527,Koné,-21.0595,164.8658,26.17,65,0,1.66,NC,1683414224
539,539,Nirmāli,26.3167,86.5833,26.63,33,0,1.42,IN,1683414253


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = sorted_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
74,Inhambane,MZ,-23.8650,35.3833,81,
341,Dibba Al-Hisn,AE,25.6196,56.2729,43,
407,Mymensingh,BD,24.7500,90.4000,72,
468,Manicoré,BR,-5.8092,-61.3003,89,
478,Parauapebas,BR,-6.0675,-49.9022,88,
499,Unaí,BR,-16.3575,-46.9061,53,
527,Koné,NC,-21.0595,164.8658,65,
539,Nirmāli,IN,26.3167,86.5833,33,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000 
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")


# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
        longitude = row["Lng"]
        latitude = row["Lat"]
 
    
        # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
        params["filter"] =  f"circle:{longitude},{latitude},{radius}"
        params["bias"] = f"proximity:{longitude},{latitude}"
    
        # Set base URL
        base_url = "https://api.geoapify.com/v2/places"


        # Make and API request using the params dictionaty
        name_address = requests.get(base_url, params = params)
    
        # Convert the API response to JSON format
        name_address = name_address.json()
    
        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
        # Log the search results
        print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Inhambane - nearest hotel: Hotel de Inhambane
Dibba Al-Hisn - nearest hotel: No hotel found
Mymensingh - nearest hotel: Hotel Amir International
Manicoré - nearest hotel: No hotel found
Parauapebas - nearest hotel: Hotel Igarapé
Unaí - nearest hotel: Hotel Fazenda Curva do Rio
Koné - nearest hotel: Pacifik Appartels
Nirmāli - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
74,Inhambane,MZ,-23.8650,35.3833,81,Hotel de Inhambane
341,Dibba Al-Hisn,AE,25.6196,56.2729,43,No hotel found
407,Mymensingh,BD,24.7500,90.4000,72,Hotel Amir International
468,Manicoré,BR,-5.8092,-61.3003,89,No hotel found
478,Parauapebas,BR,-6.0675,-49.9022,88,Hotel Igarapé
499,Unaí,BR,-16.3575,-46.9061,53,Hotel Fazenda Curva do Rio
527,Koné,NC,-21.0595,164.8658,65,Pacifik Appartels
539,Nirmāli,IN,26.3167,86.5833,33,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)